In [36]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr

# 读取数据集

本次数据分为 train.csv 和 test.csv。每个文件有10列，前9列为特征 （都为离散型），最后一列是标签（±1）。

In [3]:
def loadDataSet(filePath):
    ''' 数据集读取函数'''
    data, label = [], []
    # 读取数据集
    with open(filePath) as f:
        for line in f.readlines():
            temp = line.strip().split(",")
            data.append([float(i) for i in temp[:-1]])
            if temp[-1] != '?':
                temp[-1] = float(temp[-1])
            label.append(temp[-1])
    ##### 输出数据集相关信息 ##########
    print("data dimension of dataset：", len(data[0]))
    print("number of sample in data :", len(data))
    print("label frequency:", dict(Counter(label)))
    ##### 输出数据集相关信息 ##########
    return np.array(data), np.array(label)

In [4]:
trainSet, trainSet_label = loadDataSet('.\\data\\train.csv')
trainSet[0:5]

data dimension of dataset： 9
number of sample in data : 787
label frequency: {1.0: 323, -1.0: 464}


array([[ 23.,   1.,   2.,   2.,   1.,   1.,   2.,   0.,   0.],
       [ 36.,   1.,   1.,   5.,   1.,   1.,   2.,   3.,   0.],
       [ 33.,   1.,   3.,   3.,   1.,   1.,   2.,   3.,   0.],
       [ 35.,   3.,   3.,   2.,   1.,   0.,   0.,   3.,   0.],
       [ 25.,   1.,   2.,   4.,   1.,   1.,   2.,   2.,   0.]])

In [5]:
testSet, testSet_lable = loadDataSet('.\\data\\test.csv')
testSet[0:5]

data dimension of dataset： 9
number of sample in data : 300
label frequency: {'?': 300}


array([[ 32.,   3.,   3.,   3.,   0.,   1.,   0.,   3.,   0.],
       [ 24.,   1.,   2.,   3.,   1.,   1.,   1.,   2.,   0.],
       [ 32.,   3.,   3.,   2.,   1.,   0.,   0.,   2.,   0.],
       [ 43.,   3.,   3.,   5.,   1.,   1.,   0.,   3.,   0.],
       [ 46.,   3.,   3.,   1.,   0.,   1.,   0.,   3.,   0.]])

# 特征选取

## 信息增益和信息增益率

In [68]:
def calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=False):
    '''计算数据集每一列（特征）的信息增益或信息增益率'''
    def calcEntropy(data):
        '''计算单列数据的熵'''
        probs_ = np.array(list(Counter(data).values()))/data.shape[0]
        ans = -1*(probs_*np.log2(probs_)).sum()
        return ans
    #计算数据集的熵
    dataSetEntropy = calcEntropy(label)
    #得到数据集的 样本数 和 特征数
    sampleNum, featureNum = dataSet.shape
    infoGains = np.zeros(featureNum) #用于保存 信息增益的数组
    #对于数据集的每一个特征
    for featureId in range(featureNum):
        #得到当前的 特征
        curFeature = dataSet[:, featureId]
        #得到每个取值的统计次数
        counter = Counter(curFeature)
        #得到所有可能的取值
        values = list(counter.keys())
        #得到所有可能取值的概率
        probs = np.array(list(counter.values()))/sampleNum
        entropys = np.zeros(len(values)) #用于保存熵的数组
        #遍历每个可能的取值
        for index, val in enumerate(values):
            #得到 标签 中对应的 子数据集标签
            subLabel = label[np.argwhere(curFeature==val)[:,0]]
            #计算 子数据集标签 的 熵
            entropys[index] = calcEntropy(subLabel)
        #计算基于当前特征的 条件熵
        condEntropy = (probs*entropys).sum()
        #计算基于当前特征的 信息增益
        infoGains[featureId] = dataSetEntropy - condEntropy
        #若是计算 信息增益率，则要除以 当前特征的 熵 
        if calcInfoGainRate:
            denominator = calcEntropy(curFeature)
            #当 当前特征的 熵 为0时，信息增益也为0，此处避免除零错误
            if denominator != 0:
                infoGains[featureId] /= calcEntropy(curFeature)
    return infoGains

def calcInfoGain(dataSet, label):
    '''计算数据集每一列（特征）的信息增益'''
    return calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=False)

def calcInfoGainRate(dataSet, label):
    '''计算数据集每一列（特征）的信息增益率'''
    return calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=True)

### 测试信息增益计算结果的正确性

使用训练集进行测试，与其他同学的结果进行对比后，可知如下结果是正确的。

In [13]:
ansA = calcInfoGain(trainSet, trainSet_label)
print("每个特征的计算值如下：")
ansA
print("选取特征的下标（从0开始）为：", np.argmax(ansA) )

每个特征的计算值如下：


array([ 0.08227668,  0.01368315,  0.0152763 ,  0.10408631,  0.00169982,
        0.00107348,  0.00518738,  0.00597883,  0.00918736])

选取特征的下标（从0开始）为： 3


### 测试信息增益率计算结果的正确性

使用训练集进行测试，与其他同学的结果进行对比后，可知如下结果是正确的。

In [30]:
ansB = calcInfoGainRate(trainSet, trainSet_label)
print("每个特征的计算值如下：")
ansB
print("选取特征的下标（从0开始）为：", np.argmax(ansB) )

每个特征的计算值如下：


array([ 0.01673974,  0.00701169,  0.00964848,  0.03349103,  0.00286384,
        0.00129432,  0.00321961,  0.00328779,  0.01890093])

选取特征的下标（从0开始）为： 3


## 基尼指数

In [9]:
def calcGiniIndex(dataSet, label):
    '''计算数据集每一列（特征）的Gini指数'''
    #得到数据集的 样本数 和 特征数
    sampleNum, featureNum = dataSet.shape
    giniIndexs = np.zeros(featureNum) #用于保存 信息增益的数组
    #对于数据集的每一个特征
    for featureId in range(featureNum):
        #得到当前的 特征
        curFeature = dataSet[:, featureId]
        #得到每个取值的统计次数
        counter = Counter(curFeature)
        #得到所有可能的取值
        values = list(counter.keys())
        #得到所有可能取值的概率
        probs = np.array(list(counter.values()))/sampleNum
        subGiniIndexs = np.zeros(len(values)) #用于保存熵的数组
        #遍历每个可能的取值
        for index, val in enumerate(values):
            #得到 标签 中对应的 子数据集
            subLabel = label[np.argwhere(curFeature==val)[:,0]]
            #计算 每个取值下 数据集的 gini指数
            sub_values = np.array(list(Counter(subLabel).values()))
            sub_probs = sub_values / subLabel.shape[0]
            subGiniIndexs[index] = 1 - (sub_probs**2).sum()
        #计算基于当前特征的 gini 指数
        giniIndexs[featureId] = (probs*subGiniIndexs).sum()
    return giniIndexs

### 测试基尼指数计算结果的正确性

使用训练集进行测试，与其他同学的结果进行对比后，可知如下结果是正确的。

In [15]:
ansC = calcGiniIndex(trainSet, trainSet_label)
print("每个特征的计算值如下：")
ansC
print("选取特征的下标（从0开始）为：", np.argmax(ansC) )

每个特征的计算值如下：


array([ 0.43710387,  0.47500663,  0.47507976,  0.42602918,  0.48279887,
        0.48323448,  0.4804874 ,  0.48007143,  0.47810829])

选取特征的下标（从0开始）为： 5


## 特征选取类实现

为了更方便地使用3种特征选取，这里实现特征选取类。

In [78]:
class featureSelection:
    '''特征选取类：根据不同特征选取方法选取最优划分特征'''
    
    def __init__(self, method):
        self.method = method
        
    def getFeatureIndex(self, dataSet, label):
        '''得到最优划分属性的下标（从0开始）'''
        if self.method == 'ID3':
            return np.argmax(calcInfoGain(dataSet, label))
        elif self.method == 'C4.5':
            return np.argmax(calcInfoGainRate(dataSet, label))
        elif self.method == 'CART':
            return np.argmin(calcGiniIndex(dataSet, label))
        else:
            print("ERROR: method not define!")
        
##############测试程序###################
ID3 = featureSelection('ID3')
C45 = featureSelection('C4.5')
CART = featureSelection('CART')

a = ID3.getFeatureIndex(trainSet, trainSet_label)
b = C45.getFeatureIndex(trainSet, trainSet_label)
c = CART.getFeatureIndex(trainSet, trainSet_label)
[a,b,c]
##############测试程序###################

[3, 3, 3]

# 决策树算法

## 构建决策树的递归终止条件

假设当前结点的数据集为D，特征集为A，递归终止条件及对应的处理如下：

- 1.**D中的样本属于同一类别C，则将当前结点标记为C类叶结点**。
    - 最简单的一种递归终止条件，很好理解。


- 2.**A为空集，此时无法划分。将当前结点标记为叶结点，类别为D中出现最多的类**。
    - 比如数据集为`[['a'],['a'],['a'],['b']]`，标签为`['yes','no','yes','no']`，则经过一次划分后，特征集就为空集了。因此便无法划分下去了。而对应分类为`'a'`的标签有`['yes','no','yes']`，因此便需选择出现最多的类为结果。


- 3.**D中所有样本在A中所有特征上取值相同，此时无法划分。将当前结点标记为叶结点，类别为D中出现最多的类**。
    -  比如数据集为`[['a', 'b'],['a', 'b'],['a', 'b']]`，标签为`['yes','no','yes']`。此时数据集的两个特征的各自的取值都是一样的，此时也无法划分，因此便需选择出现最多的类为结果。
    
   
- 4.**D为空集，则将当前结点标记为叶结点，类别为父结点中出现最多的类**。
    - 这种情况在本次报告暂时还没有出现过，因此在实现中忽略该条件。


## 决策树类实现

In [83]:
class decisionTree:
    '''决策树类实现'''
    
    def __init__(self, method):
        self.featureSelectionMethod = featureSelection(method=method) #特征选取方法
    
    def __getBestSplitFeature(self, dataSet, label):
        '''得到数据集的最优划分属性的下标'''
        return self.featureSelectionMethod.getFeatureIndex(dataSet, label)
    
    def __getSubSet(self, dataSet, label, splitIndex, splitValue):
        '''根据划分属性的某个取值来得到对应的子数据集和标签'''
        #得到对应取值的样本下标
        sampleIndex = np.argwhere(dataSet[:, splitIndex]==splitValue)[:, 0]
        #得到在原数据集的基础上删除划分属性所在列对应的样本
        subDataSet = np.delete(dataSet, splitIndex, axis=1)[sampleIndex]
        #得到子标签
        subLabel = label[sampleIndex]
        return subDataSet, subLabel
    
    def __getMostCommonLabel(self, label):
        '''得到标签中出现最多次的数据'''
        return Counter(label).most_common(1)[0]
    
    def __buildTree(self, dataSet, label):
        '''递归构建决策树'''
        #递归终止条件1：若数据都属于一个类别，则返回该类别
        if len(Counter(label)) == 1:
            return label[0]
        #递归终止条件2：若遍历完所有属性，则返回标签中出现次数最多的
        if len(dataSet) == 0:
            return self.__getMostCommonLabel(label)
        #递归终止条件3：若所有样本在所有特征上取值相同，则返回标签中出现次数最多的
        check = []
        ## 遍历所有特征，得到每个特征的取值的次数
        for featureIndex in range(dataSet.shape[1]):
            check.append(len(Counter(dataSet[:, featureIndex])))
        ## 若所有特征的取值都只有一个
        if len(Counter(check)) == 1:
            return self.__getMostCommonLabel(label)
        
        #得到划分属性下标
        splitIndex = self.__getBestSplitFeature(dataSet, label)
        #以划分属性的下标为结点构建一颗空树
        tree = {splitIndex:{'tree':{}}} 
        #存储该结点对应的最可能出现的标签值（用于预测未知值）
        tree[splitIndex]['defaultLabel'] = self.__getMostCommonLabel(label)
        #遍历划分属性的所有可能取值
        for val in set(dataSet[:, splitIndex]):
            subDataSet, subLabel = self.__getSubSet(dataSet, label, splitIndex, val)
            tree[splitIndex]['tree'][val] = self.__buildTree(subDataSet, subLabel)
        return tree
        
    def buildTree(self, dataSet, label):
        '''得到决策树'''
        self.tree = self.__buildTree(dataSet, label)
    
    def __apply(self, tree, dataSet):
        '''递归应用构建好的决策树对数据集进行分类'''
        rootIndex = list(tree.keys())[0] #根结点下标
        
        
    def apply(self, dataSet):
        '''对数据集进行分类'''
        ansLabel = np.zeros(dataSet.shape[0])
        #遍历测试数据集的每一个样本
        for index, sample in enumerate(dataSet):
            ansLabel[index] = self.__apply(self.tree, sample)
        return ansLabel
        
    def getTree(self):
        '''返回训练好的以字典形式存储的决策树'''
        return self.tree

In [105]:
a = np.array([[0,1,2],[8,2,2],[3,1,5],[0,2,4]])
b = np.array(['1', '2','3no','4'])

c,d=1,1
e=np.argwhere(a[:,c]==d)[:,0]
e

np.delete(a,c,axis=1)[e]
b[e]

array([0, 2], dtype=int64)

array([[0, 2],
       [3, 5]])

array(['1', '3no'], 
      dtype='<U3')